In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#load data
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import datetime as dt
df=pd.read_csv('/kaggle/input/ecommerce-data/data.csv',encoding='latin1')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
#transfer data type
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df['CustomerID']=df['CustomerID'].astype('object')
df.dtypes

In [ ]:
#create time diff
R_today = dt.datetime(2021,8,7)
df['R_diff'] = (R_today - df['InvoiceDate']).dt.days
df.head(3)

In [ ]:
#create R,F,M
R = df.groupby(by=['CustomerID'])['R_diff'].agg([('R','min')])

In [ ]:
M=df.groupby(by=['CustomerID'])['UnitPrice'].agg([('M','sum')])

In [ ]:
#drop duplicates and calculate the recency
F_undone=df.drop_duplicates(subset='InvoiceNo')
F_undone.head(3)
F=F_undone.groupby(by=['CustomerID'])['CustomerID'].agg([('F','count')])

In [ ]:
RFM = R.join(F).join(M)
RFM.head(3)

In [ ]:
RFM.describe()

In [ ]:
#define R
RFM['R_revalue']=RFM['R']/365
RFM.head(5)

In [ ]:
#weight given: R(0.1)/F(0.5)/M(0.4)
RFM['value']=RFM['R_revalue']*0.1+RFM['F']*0.5+RFM['M']*0.4
RFM.head(3)

In [ ]:
RFM['value'].describe()
#most of the value under 124
#then we check the distribution of value which under 150

In [ ]:
RFM_Common=RFM[RFM['value']<150]
sns.set()
sns.distplot(RFM_Common['value'],bins=8)

In [ ]:
RFM_Common['value'].std()

In [ ]:
RFM_Common['value'].mean()

In [ ]:
#set four levels of customers, 'level 4' are most valuable 
bins=[0,25,50,136,16569]
RFM['type']=pd.cut(RFM['value'],bins,labels=['level 1','level 2','level 3','level 4'])

In [ ]:
RFM['type'].value_counts().plot(kind='pie',autopct='%1.2f%%',subplots=True)